In [265]:
# numpy experimentation
a=np.ones((10))
b=np.zeros(10)
b[1]=1
print a.shape, " ", b.shape
c =1.2*np.outer(a,b)
print (a*b)
print np.square(c)
print c
print c[:,1].shape
a[0][b==0]=4
print a

(10,)   (10,)
[ 0.  1.  0.  0.  0.  0.  0.  0.  0.  0.]
[[ 0.    1.44  0.    0.    0.    0.    0.    0.    0.    0.  ]
 [ 0.    1.44  0.    0.    0.    0.    0.    0.    0.    0.  ]
 [ 0.    1.44  0.    0.    0.    0.    0.    0.    0.    0.  ]
 [ 0.    1.44  0.    0.    0.    0.    0.    0.    0.    0.  ]
 [ 0.    1.44  0.    0.    0.    0.    0.    0.    0.    0.  ]
 [ 0.    1.44  0.    0.    0.    0.    0.    0.    0.    0.  ]
 [ 0.    1.44  0.    0.    0.    0.    0.    0.    0.    0.  ]
 [ 0.    1.44  0.    0.    0.    0.    0.    0.    0.    0.  ]
 [ 0.    1.44  0.    0.    0.    0.    0.    0.    0.    0.  ]
 [ 0.    1.44  0.    0.    0.    0.    0.    0.    0.    0.  ]]
[[ 0.   1.2  0.   0.   0.   0.   0.   0.   0.   0. ]
 [ 0.   1.2  0.   0.   0.   0.   0.   0.   0.   0. ]
 [ 0.   1.2  0.   0.   0.   0.   0.   0.   0.   0. ]
 [ 0.   1.2  0.   0.   0.   0.   0.   0.   0.   0. ]
 [ 0.   1.2  0.   0.   0.   0.   0.   0.   0.   0. ]
 [ 0.   1.2  0.   0.   0.   0.   0.   0.   0.   

TypeError: 'numpy.float64' object does not support item assignment

In [197]:
import numpy as np
import struct
import sys

def read_file(filename):
    with open(filename,'rb') as fp:
        zero, data_type, dims = struct.unpack('>HBB', fp.read(4))
        shape = tuple(struct.unpack('>I', fp.read(4))[0] for d in range(dims))
        np_array = np.frombuffer(fp.read(), dtype=np.uint8).reshape(shape)
    return np_array

def preprocess(image_file, label_file):
    images = read_file(image_file)
    labels = read_file(label_file)
    if (len(labels) > 10000):
        labels = labels[:10000]
        images = images[:10000]    
    images = images/255.0
    images = images.reshape( (10000, 784))

    labels = labels.reshape(-1,1)
    data = np.concatenate((images, labels), axis=1)
    np.random.shuffle(data)
    return data

def get_features_labels(data, bias):
    examples = data[:,:-1]
    labels = data[:,-1]
    classifier_labels = np.zeros((10, len(labels)))
    for i in range(10):
        classifier_labels[i][labels == i] = 1
    examples = np.append(examples, bias, 1)
    return examples, labels, classifier_labels

def get_true_label(digit, perceptron_type):
    if digit == perceptron_type:
        return 1
    return 0

In [206]:
def inference(test_data, weights):
    data_size = len(test_data)
    bias = np.ones((data_size,1))
    examples, labels, classifier_labels = get_features_labels(test_data, bias)
    prediction = np.ones(data_size, dtype = int)
    correct = 0
    for i, example in enumerate(examples):
        activation_values = sigmoid(np.sum(weights*example, axis = 1))
        prediction[i] = np.argmax(activation_values)
        if prediction[i] == labels[i]:
            correct += 1
    accuracy = correct*1.0/data_size
    return prediction, labels, accuracy

In [243]:
from numpy import linalg as LA
def sigmoid(value):
    value[value > 100] = 100
    value[value < -100] = -100
    return 1/(1+np.exp(-value))

In [66]:
path = "../hw2/DATA_FOLDER/"

train_data = preprocess(path + '/train-images.idx3-ubyte', path + '/train-labels.idx1-ubyte')
# print train_data[1]
test_data = preprocess(path + '/t10k-images.idx3-ubyte', path + '/t10k-labels.idx1-ubyte')



In [118]:
def propagate(w, X, Y):

    m = X.shape[1]

    A = sigmoid(np.dot(w,X.T)) 
    print A.shape

    cost = -1/m * np.sum(Y * np.log(A) + (1-Y) * (np.log(1-A)))

    dz= (1/m)*(A - Y)
    dw = np.dot(dz,X)
    print dw.shape

    cost = np.squeeze(cost)
    grads = {"dw": dw}

    return grads, cost

In [ ]:
def train_sgd(train_data, num_epoches, learning_rate, lamda, test_data):
    data_size = len(train_data)
    weights = np.random.uniform(-0.1,0.1,[10,785])
    bias = np.ones((data_size,1))
    for epoch in range(num_epoches):
        loss = np.zeros(10)
        np.random.shuffle(train_data)
        examples, labels, classifier_labels = get_features_labels(train_data, bias)
        delta_weights = np.zeros((10,785))
        for i,example in enumerate(examples):
            z = np.sum(weights*example, axis = 1)
            y_pred = sigmoid(z)
            delta_weights += learning_rate*np.outer(y_pred-classifier_labels[:,i],example)
            loss+=classifier_labels[:,i]*np.log(y_pred+0.00000001)+(1-classifier_labels[:,i])*np.log(1-y_pred+0.00000001)
        
        weights -= delta_weights - lamda*weights
        loss /= -data_size*1.0 + lamda
        loss = np.sum(loss)
        train_prediction, train_labels, train_accuracy = inference(train_data[:train_data_size], weights)
        test_prediction, test_labels, test_accuracy = inference(test_data, weights)
        print "epoch ", epoch, ": Training loss: ", loss, " Training Accuracy: ", train_accuracy, " Test Accuracy: ", test_accuracy
#         if loss < 3.4 and epoch > 50:
#             return weights, loss
    return weights, loss

In [283]:
def train_gd(train_data, num_epoches, learning_rate, lamda, test_data):
    data_size = len(train_data)
    weights = np.random.uniform(-0.1,0.1,[10,785])
    bias = np.ones((data_size,1))
    for epoch in range(num_epoches):
        loss = np.zeros(10)
        np.random.shuffle(train_data)
        examples, labels, classifier_labels = get_features_labels(train_data, bias)
        delta_weights = np.zeros((10,785))
        for i,example in enumerate(examples):
            z = np.sum(weights*example, axis = 1)
            y_pred = sigmoid(z)
            delta_weights += learning_rate*np.outer(y_pred-classifier_labels[:,i],example)
            loss+=classifier_labels[:,i]*np.log(y_pred+0.00000001)+(1-classifier_labels[:,i])*np.log(1-y_pred+0.00000001)
        
        weights = weights - delta_weights - lamda*weights
        loss = loss*-1.0/data_size + lamda*np.sum(np.square(weights), axis = 1)/2.0
        loss = np.sum(loss)/10.0
        train_prediction, train_labels, train_accuracy = inference(train_data[:train_data_size], weights)
        test_prediction, test_labels, test_accuracy = inference(test_data, weights)
        print "epoch ", epoch, ": Training loss: ", loss, " Training Accuracy: ", train_accuracy, " Test Accuracy: ", test_accuracy
#         if loss < 3.4 and epoch > 50:
#             return weights, loss
    return weights, loss 

In [ ]:
train_data_size = 10000
epochs = 200
learning_rate = 0.00007
lamda = 0.000001
gd_weights, train_loss = train_gd(train_data[:10000], epochs, learning_rate, lamda, test_data)
test_prediction, test_labels, test_accuracy = inference(test_data, gd_weights)
print "test accuracy ", test_accuracy, " training loss: ", train_loss

epoch  0 : Training loss:  0.734175288656  Training Accuracy:  0.2577  Test Accuracy:  0.2546
epoch  1 : Training loss:  0.953996496051  Training Accuracy:  0.5281  Test Accuracy:  0.5122
epoch  2 : Training loss:  0.627161918212  Training Accuracy:  0.5884  Test Accuracy:  0.5668
epoch  3 : Training loss:  0.335214059157  Training Accuracy:  0.6712  Test Accuracy:  0.6639
epoch  4 : Training loss:  0.215818033532  Training Accuracy:  0.7458  Test Accuracy:  0.7377
epoch  5 : Training loss:  0.177654883886  Training Accuracy:  0.7893  Test Accuracy:  0.7863
epoch  6 : Training loss:  0.159916402452  Training Accuracy:  0.8011  Test Accuracy:  0.8019
epoch  7 : Training loss:  0.153935614887  Training Accuracy:  0.8086  Test Accuracy:  0.8071
epoch  8 : Training loss:  0.148818043724  Training Accuracy:  0.8181  Test Accuracy:  0.8166
epoch  9 : Training loss:  0.144038116419  Training Accuracy:  0.8244  Test Accuracy:  0.8203
epoch  10 : Training loss:  0.140676538624  Training Accurac

epoch  87 : Training loss:  0.0847917239657  Training Accuracy:  0.8954  Test Accuracy:  0.8889
epoch  88 : Training loss:  0.0846097737776  Training Accuracy:  0.8955  Test Accuracy:  0.8892
epoch  89 : Training loss:  0.0844306951835  Training Accuracy:  0.8959  Test Accuracy:  0.8893
epoch  90 : Training loss:  0.0842544097292  Training Accuracy:  0.8962  Test Accuracy:  0.8894
epoch  91 : Training loss:  0.084080841942  Training Accuracy:  0.8963  Test Accuracy:  0.8897
epoch  92 : Training loss:  0.0839099191875  Training Accuracy:  0.8966  Test Accuracy:  0.8898
epoch  93 : Training loss:  0.0837415715345  Training Accuracy:  0.8969  Test Accuracy:  0.89
epoch  94 : Training loss:  0.0835757316274  Training Accuracy:  0.8971  Test Accuracy:  0.8899
epoch  95 : Training loss:  0.0834123345664  Training Accuracy:  0.8972  Test Accuracy:  0.8899
epoch  96 : Training loss:  0.0832513177934  Training Accuracy:  0.8975  Test Accuracy:  0.8898
epoch  97 : Training loss:  0.0830926209858

epoch  172 : Training loss:  0.0752163797146  Training Accuracy:  0.9073  Test Accuracy:  0.898
epoch  173 : Training loss:  0.0751446249112  Training Accuracy:  0.9073  Test Accuracy:  0.8981
epoch  174 : Training loss:  0.0750734233904  Training Accuracy:  0.9075  Test Accuracy:  0.8981
epoch  175 : Training loss:  0.0750027675301  Training Accuracy:  0.9078  Test Accuracy:  0.898
epoch  176 : Training loss:  0.074932649858  Training Accuracy:  0.9078  Test Accuracy:  0.8981
epoch  177 : Training loss:  0.0748630630478  Training Accuracy:  0.9078  Test Accuracy:  0.8983
epoch  178 : Training loss:  0.0747939999152  Training Accuracy:  0.9081  Test Accuracy:  0.8984
epoch  179 : Training loss:  0.0747254534149  Training Accuracy:  0.9082  Test Accuracy:  0.8985
epoch  180 : Training loss:  0.0746574166366  Training Accuracy:  0.9087  Test Accuracy:  0.8986
epoch  181 : Training loss:  0.0745898828019  Training Accuracy:  0.9089  Test Accuracy:  0.8987
epoch  182 : Training loss:  0.07

In [71]:
a= np.zeros((10,1))
sigmoid(a)

array([[ 0.5],
       [ 0.5],
       [ 0.5],
       [ 0.5],
       [ 0.5],
       [ 0.5],
       [ 0.5],
       [ 0.5],
       [ 0.5],
       [ 0.5]])